In [1]:
import os

import tensorflow as tf
import tensorflow.keras.backend as K
import numpy as np

gpu_on = True

if gpu_on :
    os.environ['CUDA_VISIBLE_DEVICES'] = '0'
    gpu_devices = tf.config.experimental.list_physical_devices("GPU")
    for device in gpu_devices:
        tf.config.experimental.set_memory_growth(device, True)
else:
    os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
    gpu_devices = tf.config.experimental.list_physical_devices("CPU")

print(gpu_devices)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
from gan_module.data_loader.classification import ClassifyDataloader
from glob import glob

data_set_name = "detect_lvi"
batch_size = 2

train_image_path = glob(f"./datasets/{data_set_name}/train/*/*")
valid_image_path = glob(f"./datasets/{data_set_name}/valid/*/*")
label_list = os.listdir(f"./datasets/{data_set_name}/train")
label_to_index_dict = {label:index for index, label in enumerate(label_list)}
index_to_label_dict = {index:label for index, label in enumerate(label_list)}
train_data_loader = ClassifyDataloader(image_path_list=train_image_path,
                                       label_to_index_dict = label_to_index_dict,
                                       batch_size = batch_size,
                                       shuffle=True
)
valid_data_loader = ClassifyDataloader(image_path_list=valid_image_path,
                                       label_to_index_dict = label_to_index_dict,
                                       batch_size = batch_size,
                                       shuffle=True
)

In [3]:
from tensorflow.keras.applications import InceptionV3, ResNet50V2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.activations import sigmoid
import numpy as np
np.random.seed(1337)  # for reproducibility

DROPOUT_RATIO = 0.5

transfer_learning = True
train_mode = "include_depp_layer"

# create the base pre-trained model
base_model = InceptionV3(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=train_data_loader.source_data_shape,
    classes=None,
    pooling=None,
    classifier_activation=None
)

if transfer_learning:
    if train_mode == "dense_only":
        base_model.trainable = False
    elif train_mode == "include_depp_layer":
        for layer in base_model.layers: 
            layer.trainable = False
            if layer.name == "mixed4":
                break

# add a global spatial average pooling layer
x = base_model.output
# (Batch_Size,?)
x = GlobalAveragePooling2D()(x)
x = Dropout(DROPOUT_RATIO)(x)
# let's add a fully-connected layer
# (Batch_Size,1)
x = Dense(1024, activation='relu')(x)
# (Batch_Size,1024)
x = Dropout(DROPOUT_RATIO)(x)
############## MODIFIED!!!!!!! 1 => 2 ##################
predictions = Dense(2, activation='sigmoid')(x)


# this is the model we will train
model = Model(base_model.input, predictions)

In [4]:
model.load_weights("./U_net/LVI_detect_classfication/weights_1.0_1.0_InceptionV3_Imagenet_LVI_level2/weights_111_0.0008.hdf5", by_name=True, skip_mismatch=True)

In [6]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.optimizers import Nadam

loss_function = tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.01)
optimizer = Nadam(1e-3, clipnorm=1)

# save_c = ModelCheckpoint(
#     "./U_net/weights_{epoch:02d}_{loss:.4f}.hdf5",
#     monitor='val_loss',
#     verbose=0,
#     save_best_only=False,
#     save_weights_only=True,
#     mode='min')

save_c = ModelCheckpoint(
    "./U_net/weights_{epoch:02d}_{loss:.4f}.hdf5",
    monitor='loss',
    verbose=0,
    save_best_only=False,
    save_weights_only=True,
    mode='min')


reduceLROnPlat = ReduceLROnPlateau(
    monitor="loss",
    factor=0.1,
    patience=10,
    verbose=1,
    mode="auto",
    min_delta=0.0001,
    cooldown=5,
    min_lr=1e-9)
csv_logger = CSVLogger('./U_net/log.csv', append=False, separator=',')

def make_model_trainable(epoch, target_epoch):
    if epoch > target_epoch:
        model.trainable = True
        
make_trainable_callback = LambdaCallback(
    on_epoch_begin=lambda epoch,logs: make_model_trainable(epoch, target_epoch=10)
)


model.compile(optimizer=optimizer, loss=loss_function, metrics=['accuracy'])

In [ ]:
start_epoch = 65
epochs = 200

model.fit(
    train_data_loader,
    validation_data=valid_data_loader,
    epochs=epochs,
    callbacks=[reduceLROnPlat, save_c, csv_logger, make_trainable_callback],
    initial_epoch=start_epoch
)

Epoch 66/200
4304/4304 [==============================] - 794s 185ms/step - loss: 0.0378 - accuracy: 1.0000 - val_loss: 1.2237 - val_accuracy: 0.6144
Epoch 67/200
4304/4304 [==============================] - 791s 184ms/step - loss: 0.0379 - accuracy: 0.9999 - val_loss: 1.2701 - val_accuracy: 0.6026
Epoch 68/200
4304/4304 [==============================] - 790s 184ms/step - loss: 0.0377 - accuracy: 1.0000 - val_loss: 1.1726 - val_accuracy: 0.6244
Epoch 69/200
4304/4304 [==============================] - 790s 184ms/step - loss: 0.0380 - accuracy: 1.0000 - val_loss: 1.1682 - val_accuracy: 0.6188
Epoch 70/200
4304/4304 [==============================] - 791s 184ms/step - loss: 0.0376 - accuracy: 1.0000 - val_loss: 1.2230 - val_accuracy: 0.5951
Epoch 71/200
4304/4304 [==============================] - 791s 184ms/step - loss: 0.0381 - accuracy: 0.9999 - val_loss: 1.1682 - val_accuracy: 0.6070
Epoch 72/200
4304/4304 [==============================] - 790s 184ms/step - loss: 0.0377 - accuracy:

4304/4304 [==============================] - 750s 174ms/step - loss: 0.0371 - accuracy: 1.0000 - val_loss: 1.3094 - val_accuracy: 0.5840
Epoch 120/200
4304/4304 [==============================] - 750s 174ms/step - loss: 0.0372 - accuracy: 1.0000 - val_loss: 1.3121 - val_accuracy: 0.5871
Epoch 121/200
4304/4304 [==============================] - 750s 174ms/step - loss: 0.0371 - accuracy: 1.0000 - val_loss: 1.2454 - val_accuracy: 0.6032
Epoch 122/200
4304/4304 [==============================] - 750s 174ms/step - loss: 0.0372 - accuracy: 0.9999 - val_loss: 1.3601 - val_accuracy: 0.5777
Epoch 123/200
4304/4304 [==============================] - 753s 175ms/step - loss: 0.0371 - accuracy: 1.0000 - val_loss: 1.2541 - val_accuracy: 0.6014
Epoch 124/200
4304/4304 [==============================] - 750s 174ms/step - loss: 0.0371 - accuracy: 1.0000 - val_loss: 1.3121 - val_accuracy: 0.5846
Epoch 125/200
4304/4304 [==============================] - 750s 174ms/step - loss: 0.0376 - accuracy: 0.9998

Epoch 172/200
4304/4304 [==============================] - 750s 174ms/step - loss: 0.0371 - accuracy: 1.0000 - val_loss: 1.3151 - val_accuracy: 0.5846
Epoch 173/200
4304/4304 [==============================] - 751s 174ms/step - loss: 0.0371 - accuracy: 1.0000 - val_loss: 1.3437 - val_accuracy: 0.5790
Epoch 174/200
2791/4304 [==================>...........] - ETA: 4:01 - loss: 0.0370 - accuracy: 1.0000